In [1]:
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm_notebook as tqdm
import glob, os
import numpy as np


In [2]:
path = "/home/koke_cacao/Documents/Koke_Cacao/Python/WorkSpace/RedstoneTorch/data/oracle_dataset/generated/*.png"
to_path = "/home/koke_cacao/Documents/Koke_Cacao/Python/WorkSpace/RedstoneTorch/data/oracle_dataset/processed/"
size = (16, 16)

In [3]:
def process_images(path, to_path, size=(32, 32), resample=Image.NEAREST, overwrite=False):
    pbar = tqdm(glob.glob(path))
    to_path = to_path + "{}/".format(size[0])
    if not os.path.exists(to_path): os.makedirs(to_path)
    
    for infile in pbar:
        file, ext = os.path.splitext(infile)
        name = file.split("/")[-1]
        im = Image.open(infile)
#         im = change_contrast(im, 200)
#         im = ImageOps.invert(im)
        im = binarize_image(im)
#         im = cap(im)
#         im = expand_boarder(im)
        im.thumbnail(size, resample)
        im.convert('1')
        if not overwrite:
            im.save(to_path + name + ".png", "PNG")
        else:
            im.save(to_path + name + ".png", "PNG")
            os.remove(infile)
            
def change_contrast(img, level):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        value = 128 + factor * (c - 128)
        return max(0, min(255, value))
    return img.point(contrast)

def cap(img):
    numpy_img = np.array(img)
#     print(numpy_img)
    numpy_img = (numpy_img < 255).astype(np.uint8)*255
    print(numpy_img)
    return Image.fromarray(numpy_img)
    

def expand_boarder(img, pixel=1):
    numpy_img = np.array(img)
    x, y, channel = numpy_img.shape
    for xs in range(x):
        for ys in range(y):
            for diff in range(pixel):
                diff = diff+1
                
                try:
                    if numpy_img[xs+diff, ys, 0] < 1:
                        numpy_img[xs, ys, 0] = 0
                        numpy_img[xs, ys, 1] = 0
                        numpy_img[xs, ys, 2] = 0
                except IndexError as e:
                    pass
                
                try:
                    if numpy_img[xs-diff, ys, 0] < 1:
                        numpy_img[xs, ys, 0] = 0
                        numpy_img[xs, ys, 1] = 0
                        numpy_img[xs, ys, 2] = 0
                except IndexError as e:
                    pass
                
                try:
                    if numpy_img[xs, ys+diff, 0] < 1:
                        numpy_img[xs, ys, 0] = 0
                        numpy_img[xs, ys, 1] = 0
                        numpy_img[xs, ys, 2] = 0
                except IndexError as e:
                    pass
                
                try:
                    if numpy_img[xs, ys-diff, 0] < 1:
                        numpy_img[xs, ys, 0] = 0
                        numpy_img[xs, ys, 1] = 0
                        numpy_img[xs, ys, 2] = 0
                except IndexError as e:
                    pass
                    
    return Image.fromarray(numpy_img)

def binarize_image(image, threshold=200):
    image = image.convert('L')
    numpy_array = np.array(image)
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] > threshold:
                numpy_array[i][j] = 255
            else:
                numpy_array[i][j] = 0
    return Image.fromarray(numpy_array)

In [6]:
process_images(path, to_path, (32, 32))